## Importing necessary packages and data

In [1]:
import numpy as np
import pandas as pd

## 1. Stylized Duration and Convexity

### 1.1. Calculate duration

**Parameters:**
* Face Value ($F$): $100
* Time to Maturity ($T$): 10 years
* Coupon Rate ($c$): 3% (annual)
* Yield to Maturity ($y$): 5% (annual, semiannual compounding)
* Frequency ($m$): 2 (semiannual)

**Formulas:**
The cash flows occur at $t = 0.5, 1.0, \dots, 10.0$.
The periodic yield is $r = y/m = 2.5\%$.
The periodic coupon is $C = (c \times F) / m = \$1.50$.

The **Price ($P$)** is the present value of cash flows:
$$P = \sum_{n=1}^{N} \frac{CF_n}{(1+r)^n}$$

**Macaulay Duration ($D_{mac}$)** (in years):
$$D_{mac} = \frac{1}{P} \sum_{n=1}^{N} \frac{n}{m} \times \frac{CF_n}{(1+r)^n}$$

**Modified Duration ($D_{mod}$)**:
$$D_{mod} = \frac{D_{mac}}{1+r}$$

In [2]:
def calculate_duration_metrics(face_value=100, coupon_rate=0.03, ytm=0.05, years=10, freq=2):
    periods = years * freq
    r_periodic = ytm / freq
    c_periodic = (coupon_rate * face_value) / freq
    
    # Time vector (0.5, 1.0, ..., 10.0)
    t_vec = np.arange(1, periods + 1) / freq
    
    # Cash flow vector
    cf_vec = np.full(periods, c_periodic)
    cf_vec[-1] += face_value  # Add principal to last payment
    
    # Present Value of Cash Flows
    discount_factors = (1 + r_periodic) ** -(t_vec * freq)
    pv_vec = cf_vec * discount_factors
    price = np.sum(pv_vec)
    
    # Macaulay Duration (Weighted Average Time)
    weighted_time = np.sum(t_vec * pv_vec)
    mac_dur = weighted_time / price
    
    # Modified Duration
    mod_dur = mac_dur / (1 + r_periodic)
    
    return price, mac_dur, mod_dur

# 1.1 Calculation
price_1, mac_1, mod_1 = calculate_duration_metrics(coupon_rate=0.03)

# Display the results
results_df = pd.DataFrame({
    "Metric": ["Price", "Macaulay Duration", "Modified Duration"],
    "Value": [price_1, mac_1, mod_1],
    "Unit": ["USD", "years", "years"]
})

results_df = results_df.set_index("Metric")
display(results_df.style.format({"Value": "{:.6f}"}))

,Value,Unit
Metric,,
Price,84.410838,USD
Macaulay Duration,8.570879,years
Modified Duration,8.361834,years


### 1.2. Duration Calculation (10Y, 7% Coupon)

Keeping all other parameters constant ($T=10$, $y=5\%$), we change the coupon rate to 7%.

In [3]:
# Calculation
price_2, mac_2, mod_2 = calculate_duration_metrics(coupon_rate=0.07)

results_df_2 = pd.DataFrame({
    "Metric": ["Price", "Macaulay Duration", "Modified Duration"],
    "Value": [price_2, mac_2, mod_2],
    "Unit": ["USD", "years", "years"]
})
results_df_2 = results_df_2.set_index("Metric")
display(results_df_2.style.format({"Value": "{:.6f}"}))

,Value,Unit
Metric,,
Price,115.589162,USD
Macaulay Duration,7.564844,years
Modified Duration,7.380336,years


### 1.3. Duration Calculation (10Y, 5% Coupon)

Keeping all other parameters constant, we change the coupon rate to 5%.
*Note: Since Coupon Rate = Yield, the bond should trade at Par ($100).*

In [6]:
# Calculation
price_3, mac_3, mod_3 = calculate_duration_metrics(coupon_rate=0.05)

results_df_3 = pd.DataFrame({
    "Metric": ["Price", "Macaulay Duration", "Modified Duration"],
    "Value": [price_3, mac_3, mod_3],
    "Unit": ["USD", "years", "years"]
})
results_df_3 = results_df_3.set_index("Metric")
display(results_df_3.style.format({"Value": "{:.6f}"}))

,Value,Unit
Metric,,
Price,100.000000,USD
Macaulay Duration,7.989446,years
Modified Duration,7.794581,years


### 1.4. Impact of Coupon Rate on Duration

**Comparison of Results:**
* **3% Coupon:** Modified Duration $\approx 8.36$ years
* **5% Coupon:** Modified Duration $\approx 7.79$ years
* **7% Coupon:** Modified Duration $\approx 7.38$ years

**Explanation:**
There is an **inverse relationship** between the coupon rate and duration. As the coupon rate increases, the duration decreases.

**Economic Intuition:**
Duration represents the weighted average time to receive the bond's cash flows.
1.  **Higher Coupon:** A larger portion of the total return is received earlier in the form of coupon payments, reducing the duration.
2.  **Lower Coupon:** A larger proportion of the bond's value comes from the final principal repayment at year 10. This pushes the weighted average time out further, increasing duration.

## 2. Hedging Duration